# Tensorflow Hub Model with 0.5 Percent Data Augmenetation (Top 10 Diagnosis)

In [19]:
import pickle, random
import numpy as np

In [20]:
import pandas as pd

In [21]:
def get_sorted_dict(D):
    return {k: v for k, v in sorted(D.items(), key=lambda item: item[1], reverse=1)}

In [22]:
winogender_sentences = "all_sentences.tsv"

In [23]:
winogender_occupations = "occupations-stats.tsv"

In [24]:
import csv

wino_occupations = []

In [25]:
with open(winogender_occupations, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        wino_occupations.append(lines["occupation"])

In [26]:
# wino_occupations

In [27]:
len(wino_occupations)

60

In [28]:
len(set(wino_occupations))

60

### Load Text Classification Hub Model

In [29]:
import tensorflow as tf

In [30]:
rand_range = range(1, 11)

In [31]:
percent_mode = "half"
path_start = "../saved_model/" + str(percent_mode) + "_percent_models/my_model_txt_classifier_hub_with_" + str(percent_mode) + "_percent_extra_data_rand_"
path_start

'../saved_model/half_percent_models/my_model_txt_classifier_hub_with_half_percent_extra_data_rand_'

In [32]:
# my_model directory
# !ls ../saved_model

test_pth = "../saved_model/" + str(percent_mode)+ "_percent_models/"
!ls $test_pth

my_model_txt_classifier_hub_with_half_percent_extra_data_rand_1
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_10
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_2
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_3
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_4
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_5
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_6
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_7
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_8
my_model_txt_classifier_hub_with_half_percent_extra_data_rand_9


In [33]:
# Contains an assets folder, saved_model.pb, and variables folder.
test_pth = "../saved_model/" + str(percent_mode)+ "_percent_models/my_model_txt_classifier_hub_with_" + str(percent_mode) + "_percent_extra_data_rand_1"
!ls $test_pth

assets         saved_model.pb variables


## Test on Repaired Input Tokens

### A. Direct gender noun comparisons (e.g. My boyfriend/My girlfriend)

In [34]:
mode = "direct-gender-noun"

In [35]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [36]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [37]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [38]:
noun_error = {**noun_error1, **noun_error2}

In [39]:
# noun_error 

In [40]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [41]:
noun_error1

['my aunt',
 'my daughter',
 'my mother',
 'my wife',
 'this girl',
 'my sister',
 'this woman',
 'my girlfriend']

In [42]:
noun_error2

['my uncle',
 'my son',
 'my father',
 'my brother',
 'my husband',
 'this boy',
 'this man',
 'my boyfriend']

In [43]:
noun_error = get_sorted_dict(noun_error)

In [44]:
noun_error = list(noun_error)

In [45]:
input1, input2, input3 = [],[], []

In [46]:
input_pairs, input_truple = [], []

In [47]:
occupation_list = []

In [48]:
i = 0
tmp, tmp2 ="", ""

In [49]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [50]:
f_input, m_input, neut_input = "", "", ""

In [51]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
#             print("male_sentence: ", lines["sentence"])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
#             print("female_sentence: ", lines["sentence"])
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
#             print("female_sentence: ", lines["sentence"])
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1
#         if i >= 6:
#             break

In [52]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [53]:
# fairness_errors = []
# all_inputs = []
# count = 0

In [54]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [55]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [56]:
len(all_inputs)

3000

In [57]:
len(fairness_errors)

169

In [58]:
len(fairness_errors_all_rands)

10

In [59]:
fairness_errors_all_rands

[262, 270, 172, 178, 193, 143, 183, 207, 235, 169]

In [60]:
sum(fairness_errors_all_rands)

2012

In [61]:
sum(fairness_errors_all_rands)/10

201.2

In [62]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [63]:
sum(all_rand_inputs)

30000

In [64]:
sum(all_rand_inputs)/10

3000.0

In [65]:
A_errors = sum(fairness_errors_all_rands)

In [66]:
A_inputs = sum(all_rand_inputs)

### B. Random gender noun comparisons (e.g. My boyfriend/My mother)

In [67]:
mode = "random-gender-noun"

In [68]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [69]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [70]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [71]:
noun_error = {**noun_error1, **noun_error2}

In [72]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [73]:
noun_error1

['my aunt',
 'my girlfriend',
 'my daughter',
 'my wife',
 'my mother',
 'this woman',
 'this girl',
 'my sister',
 'my mom']

In [74]:
noun_error2

['my brother',
 'my father',
 'this boy',
 'my boyfriend',
 'my uncle',
 'my husband',
 'this man',
 'my dad',
 'my son']

In [75]:
noun_error = get_sorted_dict(noun_error)

In [76]:
noun_error = list(noun_error)

In [77]:
input1, input2, input3 = [],[], []

In [78]:
input_pairs, input_truple = [], []

In [79]:
occupation_list = []

In [80]:
i = 0
tmp, tmp2 ="", ""

In [81]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [82]:
f_input, m_input, neut_input = "", "", ""

In [83]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = ""
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [84]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [85]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [86]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [87]:
len(all_inputs)

3000

In [88]:
len(fairness_errors)

135

In [89]:
len(fairness_errors_all_rands)

10

In [90]:
fairness_errors_all_rands

[193, 243, 226, 203, 242, 116, 200, 198, 200, 135]

In [91]:
sum(fairness_errors_all_rands)

1956

In [92]:
sum(fairness_errors_all_rands)/10

195.6

In [93]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [94]:
sum(all_rand_inputs)

30000

In [95]:
sum(all_rand_inputs)/10

3000.0

In [96]:
B_errors = sum(fairness_errors_all_rands)

In [97]:
B_inputs = sum(all_rand_inputs)

### C. Test for Indirect Gender Bias, i.e. Occupational Bias

In [98]:
mode = "gender-occupation-noun"

In [99]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [100]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [101]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [102]:
noun_error = {**noun_error1, **noun_error2}

In [103]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [104]:
noun_error1

['the writer',
 'the teacher',
 'the cleaner',
 'the auditor',
 'the counselor',
 'the nurse',
 'the the paralegal',
 'the editor',
 'the tailor',
 'the clerk']

In [105]:
noun_error2

['the cook',
 'the analyst',
 'the engineer',
 'the mechanic',
 'the technician',
 'the supervisor',
 'the salesperson',
 'the farmer',
 'the chief',
 'the lawyer']

In [106]:
noun_error = get_sorted_dict(noun_error)

In [107]:
noun_error = list(noun_error)

In [108]:
input1, input2, input3 = [],[], []

In [109]:
input_pairs, input_truple = [], []

In [110]:
occupation_list = []

In [111]:
i = 0
tmp, tmp2 ="", ""

In [112]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [113]:
f_input, m_input, neut_input = "", "", ""

In [114]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" #lines["sentid"]
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [115]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [116]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [117]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [118]:
len(all_inputs)

3000

In [119]:
len(fairness_errors)

123

In [120]:
len(fairness_errors_all_rands)

10

In [121]:
fairness_errors_all_rands

[330, 392, 190, 201, 332, 243, 196, 282, 219, 123]

In [122]:
sum(fairness_errors_all_rands)

2508

In [123]:
sum(fairness_errors_all_rands)/10

250.8

In [124]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [125]:
sum(all_rand_inputs)

30000

In [126]:
sum(all_rand_inputs)/10

3000.0

In [127]:
C_errors = sum(fairness_errors_all_rands)

In [128]:
C_inputs = sum(all_rand_inputs)

### D. Test for   for Indirect Gender Bias, i.e. Name Bias

In [129]:
mode = "gender-name-noun"

In [130]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [131]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [132]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [133]:
noun_error = {**noun_error1, **noun_error2}

In [134]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [135]:
noun_error1

['Deborah',
 'Sharon',
 'Kimberly',
 'Carol',
 'Michelle',
 'Karen',
 'Sandra',
 'Laura',
 'Nancy',
 'Barbara']

In [136]:
noun_error2

['Christopher',
 'Joshua',
 'Thomas',
 'James',
 'Timothy',
 'Mark',
 'Robert ',
 'George',
 'Edward',
 'Anthony']

In [137]:
noun_error = get_sorted_dict(noun_error)

In [138]:
noun_error = list(noun_error)

In [139]:
input1, input2, input3 = [],[], []

In [140]:
input_pairs, input_truple = [], []

In [141]:
occupation_list = []

In [142]:
i = 0
tmp, tmp2 ="", ""

In [143]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [144]:
f_input, m_input, neut_input = "", "", ""

In [145]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1

In [146]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [147]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [148]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [149]:
len(all_inputs)

3000

In [150]:
len(fairness_errors)

272

In [151]:
len(fairness_errors_all_rands)

10

In [152]:
fairness_errors_all_rands

[408, 442, 269, 255, 312, 242, 347, 232, 378, 272]

In [153]:
sum(fairness_errors_all_rands)

3157

In [154]:
sum(fairness_errors_all_rands)/10

315.7

In [155]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [156]:
sum(all_rand_inputs)

30000

In [157]:
sum(all_rand_inputs)/10

3000.0

In [158]:
D_errors = sum(fairness_errors_all_rands)

In [159]:
D_inputs = sum(all_rand_inputs)

### E. Test for   for Indirect Racial Bias, i.e. Name Bias

In [160]:
mode = "racial-name-noun"

In [161]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [162]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [163]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [164]:
noun_error = {**noun_error1, **noun_error2}

In [165]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [166]:
noun_error1

['Josh',
 'Melanie',
 'Justin',
 'Stephanie',
 'Harry',
 'Jack',
 'Katie',
 'Nancy',
 'Amanda',
 'Ellen']

In [167]:
noun_error2

['Malik',
 'Lamar',
 'Jasmine',
 'Jamel',
 'Nichelle',
 'Latisha',
 'Darnell',
 'Leroy',
 'Alonzo',
 'Shereen']

In [168]:
noun_error = get_sorted_dict(noun_error)

In [169]:
noun_error = list(noun_error)

In [170]:
input1, input2, input3 = [],[], []

In [171]:
input_pairs, input_truple = [], []

In [172]:
occupation_list = []

In [173]:
i = 0
tmp, tmp2 ="", ""

In [174]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [175]:
f_input, m_input, neut_input = "", "", ""

In [176]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" #lines["sentid"]
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = ""
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [177]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [178]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [179]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [180]:
len(all_inputs)

3000

In [181]:
len(fairness_errors)

213

In [182]:
len(fairness_errors_all_rands)

10

In [183]:
fairness_errors_all_rands

[271, 245, 256, 275, 217, 230, 276, 155, 235, 213]

In [184]:
sum(fairness_errors_all_rands)

2373

In [185]:
sum(fairness_errors_all_rands)/10

237.3

In [186]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [187]:
sum(all_rand_inputs)

30000

In [188]:
sum(all_rand_inputs)/10

3000.0

In [189]:
E_errors = sum(fairness_errors_all_rands)

In [190]:
E_inputs = sum(all_rand_inputs)

### F. Test for Neutral (Sentiment) Sentences

In [191]:
mode = "neutral-sentiments"

In [192]:
pickle_path_1 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error1.pickle'

pickle_path_2 = '/Users/ezekiel.soremekun/Documents/Coref-Fairness-Test-Generation/Ezekiel-Testbed/trained-sentiment-analyzers/Exploitation/saved_pickles/exploitation/hub/'  + \
                mode + '/noun_error2.pickle'

In [193]:
with open(pickle_path_1, 'rb') as handle:
    noun_error1 = pickle.load(handle)
    noun_error1.pop('her', None)
    noun_error1.pop('She', None)
    noun_error1 = get_sorted_dict(noun_error1)

In [194]:
with open(pickle_path_2, 'rb') as handle:
    noun_error2 = pickle.load(handle)
    noun_error2.pop('him', None)
    noun_error2.pop('He', None)
    noun_error2 = get_sorted_dict(noun_error2)

In [195]:
noun_error = {**noun_error1, **noun_error2}

In [196]:
# noun_error 

In [197]:
noun_error1 = list(noun_error1)[:10]
noun_error2 = list(noun_error2)[:10]

In [198]:
noun_error1

['my aunt',
 'my daughter',
 'my wife',
 'my mother',
 'my girlfriend',
 'this girl',
 'this woman',
 'my sister',
 'my mom',
 'the writer']

In [199]:
noun_error2

['my son',
 'my uncle',
 'my father',
 'my husband',
 'my brother',
 'this boy',
 'this man',
 'my boyfriend',
 'my dad',
 'the cook']

In [200]:
noun_error = get_sorted_dict(noun_error)

In [201]:
noun_error = list(noun_error)

In [202]:
input1, input2, input3 = [],[], []

In [203]:
input_pairs, input_truple = [], []

In [204]:
occupation_list = []

In [205]:
i = 0
tmp, tmp2 ="", ""

In [206]:
male_token = ".male."
female_token = ".female."
neutral_token = ".neutral."

In [207]:
f_input, m_input, neut_input = "", "", ""

In [208]:
with open(winogender_sentences, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter='\t')
    for lines in csv_reader:
        if male_token in lines["sentid"]:
            occupation_list.append(lines["sentid"].split(".")[0])
            input1.append(lines["sentence"])
            m_input = lines["sentence"]
            tmp = lines["sentid"].replace(male_token, female_token)
            tmp2 = lines["sentid"].replace(male_token, neutral_token)
        elif (female_token in lines["sentid"]) and (lines["sentid"] == tmp):
            input2.append(lines["sentence"])
            f_input = lines["sentence"]
            tmp = "" 
        elif (neutral_token in lines["sentid"]) and (lines["sentid"] == tmp2):
            input3.append(lines["sentence"])
            neut_input = lines["sentence"]
            tmp = "" 
            
        if m_input and f_input:
            input_truple.append((f_input, m_input, neut_input))
            f_input, m_input, neut_input = "", "", ""
        i+= 1


In [209]:
occupation_list[:5]

['technician', 'technician', 'technician', 'technician', 'accountant']

In [210]:
fairness_errors_all_rands = []
all_rand_inputs = []

In [211]:
for ind in range(0, len(rand_range)):
    model_path = path_start + str(ind+1) + '/'
    hub_model = tf.keras.models.load_model(model_path)
    
    fairness_errors = []
    all_inputs = []
    count = 0
    
    for i in range(len(input_truple)):
        if occupation_list[i] in wino_occupations:
            for j in range(len(noun_error1)) :
                count += 1
                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][0]:
                    input1 = input_truple[i][0].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][0].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][0].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][1]:
                    input1 = input_truple[i][1].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][1].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][1].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   


                input1, input2 = "", ""
                if ("the " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("the " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("the " + occupation_list[i], noun_error2[j])
                elif ("The " + occupation_list[i]) in  input_truple[i][2]:
                    input1 = input_truple[i][2].replace("The " + occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace("The " + occupation_list[i], noun_error2[j])
                else:
                    input1 = input_truple[i][2].replace(occupation_list[i], noun_error1[j])
                    input2 = input_truple[i][2].replace(occupation_list[i], noun_error2[j])     
                all_inputs.append((input1, input2))
                pred1 = hub_model.predict(tf.convert_to_tensor(tf.constant([input1]).numpy(), dtype=tf.string))
                pred2 = hub_model.predict(tf.convert_to_tensor(tf.constant([input2]).numpy(), dtype=tf.string))
                if (np.sign(pred1) != np.sign(pred2)):
                    fairness_errors.append((input1, input2))   

                if len(all_inputs) >= 3000:
                    break

        if len(all_inputs) >= 3000:
            break
    fairness_errors_all_rands.append(len(fairness_errors))
    all_rand_inputs.append(len(all_inputs))


In [212]:
len(all_inputs)

3000

In [213]:
len(fairness_errors)

121

In [214]:
len(fairness_errors_all_rands)

10

In [215]:
fairness_errors_all_rands

[227, 257, 186, 214, 264, 140, 182, 192, 217, 121]

In [216]:
sum(fairness_errors_all_rands)

2000

In [217]:
sum(fairness_errors_all_rands)/10

200.0

In [218]:
all_rand_inputs

[3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000]

In [219]:
sum(all_rand_inputs)

30000

In [220]:
sum(all_rand_inputs)/10

3000.0

In [221]:
F_errors = sum(fairness_errors_all_rands)

In [222]:
F_inputs = sum(all_rand_inputs)

In [223]:
all_inputs = A_inputs + B_inputs + C_inputs + D_inputs + E_inputs + F_inputs

In [224]:
all_errors = A_errors + B_errors + C_errors + D_errors + E_errors + F_errors

In [225]:
all_inputs

180000

In [226]:
all_errors

14006

In [227]:
all_inputs/6

30000.0

In [228]:
all_errors/6

2334.3333333333335

In [229]:
all_errors_list = [A_errors, B_errors, C_errors, D_errors, E_errors, F_errors]

In [230]:
all_errors_list

[2012, 1956, 2508, 3157, 2373, 2000]

In [231]:
all_inputs_list = [A_inputs, B_inputs, C_inputs, D_inputs, E_inputs, F_inputs]

In [232]:
all_inputs_list

[30000, 30000, 30000, 30000, 30000, 30000]